In [9]:
# Importing the necessary modules
import pygame, sys, random
import numpy as np
import pandas as pd
import scipy as sp

### Defining constants
The following cell defines the constants needed to run the Pygame window and the visualiser as a whole. For example, the constant RUNNING is the value that determines whether the game loop is running.

There are other constants defined in this section such as the colours that are used throughout the program. These follow the (R,G,B,A) structure, where R, G, B refer to the amount of red, green and blue making up the colour ranging between 0-255 with 0 being no colour and 255 being maximum colour. A represents the alpha value which is how transparent the colour is, again ranging between 0-255 being set to 255 by default.

Further down, some Pygame surfaces and rectangles are created. A surface is used in Pygame to represent the appearance of an object and give it colour, with surfaces being used to generate all the buttons, text and backgrounds. The rectangles are then used to position these surfaces in the correct places, and for the buttons the rectangles are used to check for collisions.

A dictionary was also used when defining the stability of the reactor to pass the stability of the reactor as the key, and get the corresponding colour back as an output.

In [10]:
# Pygame running constants
RUNNING = True
DISPLAY_WIDTH = 1280
DISPLAY_HEIGHT = 720
CHOSEN_FPS = 15

# Colours
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
FUEL_ROD_COLOUR = (70, 130, 70)

BACKGROUND_COLOUR = (170, 170, 220)
START_SCREEN_BACKGROUND_COLOUR = (56, 209, 0)
BUTTON_COLOUR = (255, 190, 0)
BUTTON_OUTLINE_COLOUR = (80, 80, 80)

CONTROL_BUTTON_COLOUR_MINUS_0_1x = (255, 22, 0)
CONTROL_BUTTON_COLOUR_MINUS_1_0x = (224, 21, 0)
CONTROL_BUTTON_COLOUR_MINUS_5_0x = (181, 15, 0)
CONTROL_BUTTON_COLOUR_PLUS_0_1x = (2, 209, 9)
CONTROL_BUTTON_COLOUR_PLUS_1_0x = (0, 227, 8)
CONTROL_BUTTON_COLOUR_PLUS_5_0x = (0, 255, 9)

# Help button parameters
HELP_BUTTON_WIDTH = 80
HELP_BUTTON_HEIGHT = 40
helpButtonSurf = pygame.Surface((HELP_BUTTON_WIDTH, HELP_BUTTON_HEIGHT))
helpButtonRect = helpButtonSurf.get_rect(topleft=(DISPLAY_WIDTH-HELP_BUTTON_WIDTH,0))

# Start menu parameters
BUTTON_OFFSET = 250
visualiserReturnButtonSurf = pygame.Surface((HELP_BUTTON_WIDTH, HELP_BUTTON_HEIGHT))
visualiserReturnButtonRect = visualiserReturnButtonSurf.get_rect(topright=(DISPLAY_WIDTH-HELP_BUTTON_WIDTH-BUTTON_OFFSET/5,0))

#Start button
START_BUTTON_WIDTH = 220
START_BUTTON_HEIGHT = 40
startButtonSurf = pygame.Surface((START_BUTTON_WIDTH, START_BUTTON_HEIGHT))
startButtonRect = startButtonSurf.get_rect(center =(DISPLAY_WIDTH/2, DISPLAY_HEIGHT/2))

#water button
WATER_BUTTON_WIDTH = 200
WATER_BUTTON_HEIGHT = 40
waterButtonSurf = pygame.Surface((WATER_BUTTON_WIDTH, WATER_BUTTON_HEIGHT))
waterButtonRect = waterButtonSurf.get_rect(center =(DISPLAY_WIDTH/2 + BUTTON_OFFSET, DISPLAY_HEIGHT/2 +BUTTON_OFFSET))

#heavy water button
HEAVY_WATER_BUTTON_WIDTH = 200
HEAVY_WATER_BUTTON_HEIGHT = 40
heavyWaterButtonSurf = pygame.Surface((HEAVY_WATER_BUTTON_WIDTH, HEAVY_WATER_BUTTON_HEIGHT))
heavyWaterButtonRect = heavyWaterButtonSurf.get_rect(center =(DISPLAY_WIDTH/2, DISPLAY_HEIGHT/2 +BUTTON_OFFSET))

#graphite button 
GRAPH_BUTTON_WIDTH = 200
GRAPH_BUTTON_HEIGHT = 40
graphButtonSurf = pygame.Surface((GRAPH_BUTTON_WIDTH, GRAPH_BUTTON_HEIGHT))
graphButtonRect = graphButtonSurf.get_rect(center =(DISPLAY_WIDTH/2 -BUTTON_OFFSET, DISPLAY_HEIGHT/2 +BUTTON_OFFSET))

# Control rod button parameters
CONTROL_BUTTON_SIZE = 55
controlButtonSurf = pygame.Surface((CONTROL_BUTTON_SIZE,CONTROL_BUTTON_SIZE))
controlButtonRect_minus_0_1 = controlButtonSurf.get_rect(topleft = (1*CONTROL_BUTTON_SIZE/2, 1.5*CONTROL_BUTTON_SIZE/2))
controlButtonRect_minus_1_0 = controlButtonSurf.get_rect(topleft = (4*CONTROL_BUTTON_SIZE/2, 1.5*CONTROL_BUTTON_SIZE/2))
controlButtonRect_minus_5_0 = controlButtonSurf.get_rect(topleft = (7*CONTROL_BUTTON_SIZE/2, 1.5*CONTROL_BUTTON_SIZE/2))
controlButtonRect_plus_0_1 = controlButtonSurf.get_rect(topleft = (1*CONTROL_BUTTON_SIZE/2, 4*CONTROL_BUTTON_SIZE/2))
controlButtonRect_plus_1_0 = controlButtonSurf.get_rect(topleft = (4*CONTROL_BUTTON_SIZE/2, 4*CONTROL_BUTTON_SIZE/2))
controlButtonRect_plus_5_0 = controlButtonSurf.get_rect(topleft = (7*CONTROL_BUTTON_SIZE/2, 4*CONTROL_BUTTON_SIZE/2))

# Control Rod parameters
CONTROL_ROD_WIDTH = 5
CONTROL_ROD_HEIGHT = 240
CONTROL_ROD_OFFSET = 21.4
NUMBER_OF_CONTROL_RODS = 8

# Fuel Rod parameters
FUEL_ROD_WIDTH = 7
FUEL_ROD_HEIGHT = 200
FUEL_ROD_OFFSET = 21.4
NUMBER_OF_FUEL_RODS = 9

# The constants which define the stability box
stabilityBoxDimensions = (172, 96.75)
stabilityBoxCoords = (1098, 226)

# A dictionary that returns the colour corresponding to the current stability when passed a string reference.
stabilityColours = {
    "Unstable": (238, 24, 24),
    "Stable": (60, 242, 86),
    "Dying out": (224, 224, 224)
}

### Defining the classes
It was chosen to take an Object-Oriented approach to creating the control and fuel rods. This was so that several identical rods could be made with just their position different in a short amount of lines.

In [11]:
# http://www.codingwithruss.com/pygame/sprite-class-and-sprite-groups-explained/#:~:text=Python%20lists%20are%20an%20obvious,around%20and%20checking%20for%20collisions.
# The previous link was used to refresh my memory on the basics of Pygame sprites, nothing was copied from this page but I will put the link here as a reference.

class ControlRod(pygame.sprite.Sprite):
    def __init__(self, width, height, position):
        '''
        Initialises the control rod object and the pygame Sprite superclass. Sets attributes like the original position and current position.
        '''
        pygame.sprite.Sprite.__init__(self) 
        self.Create(width, height)
        
        self.originalPosition = position
        self.position = position
        self.rect.topleft = self.position

    def Create(self, width, height):
        '''
        Creates the control rod by creating a surface with the passed width and height and then getting the corresponding rectangle from that surface.
        '''
        self.width, self.height = width, height
        self.image = pygame.Surface((self.width, self.height))
        self.rect = self.image.get_rect()

    def MoveUp(self, relativeShift):
        '''
        Given a relative shift, it will move the control rod up on the screen given that the movement will not take the control rod outside of the allowed range of values, 0-100.
        '''
        if self.position[1] + self.height*relativeShift/100 > self.originalPosition[1]:
            yPos = self.position[1] + self.height*relativeShift/100
            self.position = (self.position[0], yPos)
            self.rect.topleft = self.position
        if self.position[1] < self.originalPosition[1]:
            self.position = (self.originalPosition[0], self.originalPosition[1] + self.height*relativeShift/100)
            self.rect.topleft = self.position

    def MoveDown(self, relativeShift):
        '''
        Given a relative shift, it will move the control rod down on the screen given that the movement will not take the control rod outside of the allowed range of values, 0-100.
        '''
        if self.position[1] + self.height*relativeShift/100 < self.originalPosition[1] + self.height:
            yPos = self.position[1] + self.height*relativeShift/100
            self.position = (self.position[0], yPos)
            self.rect.topleft = self.position
        if self.position[1] > self.originalPosition[1] + self.height:
            self.position = (self.position[0], self.originalPosition[1] + self.height)
            self.rect.topleft = self.position

class FuelRod(pygame.sprite.Sprite):
    def __init__(self, width, height, position):
        '''
        Initialises the control rod object and the pygame Sprite superclass. Creates the surface and fills it with the correct colour, as well as getting the associated rectangle.
        '''
        pygame.sprite.Sprite.__init__(self)
        self.width, self.height = width, height
        self.image = pygame.Surface((self.width, self.height))
        self.image.fill(FUEL_ROD_COLOUR)
        self.rect = self.image.get_rect(topleft = position)

### Selecting moderator
For modelling our fission reactor, it was important to figure out the moderator and coolant that would be best to use. In order to do this it was important to get some numbers so that we could rank a few moderators to pick the best one.

First we define a function that determines the logarithmic energy loss, and this defines the amount of energy the neutron loses upon a collision with an atomic with atomic number A. This should be large so that the most amount of energy is lost per collision.

Second is a function for calculating the slowing down power. It quantifies how good a moderator is at scattering neutrons. This should be large as neutrons should be scattered as much as possible inside a moderator.

Third is a function for calculating the moderating ratio. This quantifies how good of a moderator a material is by account for the logarithmic energy loss, scattering cross section and absoption cross section. This should be as large as possible. Ideally absoprtion cross section is small for a good moderator as neutrons should not be absorbed.

Finally a function for calculating the number of collisions to thermalise is calculated. This is the number of collisions required on average to bring a neutron released from fission to thermal speeds (able to induce further fission). This should be low as less collisions required means the moderator can be thinner.

The last thing done in this code cell is writing all of the moderator details to their own dictionary and then adding these dictionaries to a Pandas dataframe to visualise the results.

In [12]:
def LogarithmicEnergyLoss(A):
    '''
    Calculates the logarithmic energy loss for a given atomic number, A. lel = logarithmic energy loss, alpha is a constant for a
    given A and makes the equations easier to read.
    '''
    alpha = ((A-1)**2)/((A+1)**2)
    lel = 1+((alpha*np.log(alpha))/(1-alpha))
    return lel


def SlowingDownPower(lel, sxs):
    '''
    Calculates the slowing down power for a given logarithmic energy loss and scattering cross section. lel = logarithmic energy loss
    sxs = scattering cross section.
    '''
    sdp = lel*sxs
    return sdp 

def ModeratingRatio(sdp,axs):
    '''
    Calculates the moderating ratio for a given slowing down power and absorption cross section. sdp = slowing down power 
    axs = absorption cross section
    '''
    mdr = sdp/axs
    return mdr

def CollisionsToThermalise(lel):
    '''
    Calculates the number of colisions to thermalise a neutron from 2MeV to 0.025eV as this is the energy the neutron will originally 
    have after a fission event. lel = logarithmic energy loss
    '''
    startEnergy = 2e6 #eV
    endEnergy = 0.025 #eV
    ncollisions = np.log((startEnergy)/endEnergy)/lel
    return ncollisions


# The data in this dictionary takes the form key:value with the key being the name of the property of the moderator, and the value being the property
# keys = AbsorptionCrossSection, ScatteringCrossSection, AtomicNumber, LogarithmicEnergyLoss, NumberOfCollisionsToThermalise, SlowingDownPower, ModeratingRatio.

waterModerator = {
    "AbsorptionCrossSection" : 2.2,
    "ScatteringCrossSection" : 165, 
    "AtomicNumber" : 16, 
}

heavyWaterModerator = {
    "AbsorptionCrossSection" : 0.012,
    "ScatteringCrossSection" : 36, 
    "AtomicNumber" : 20, 
}

graphiteModerator = {
    "AbsorptionCrossSection" : 0.003,
    "ScatteringCrossSection" : 39, 
    "AtomicNumber" : 12,
}

# Adding to water
waterModerator["LogarithmicEnergyLoss"] = 0.927 # Cannot be calculated using formula above due to being a molecule
waterModerator["NumberOfCollisionsToThermalise"] = CollisionsToThermalise(waterModerator["LogarithmicEnergyLoss"])
waterModerator["SlowingDownPower"] = SlowingDownPower(waterModerator["LogarithmicEnergyLoss"], waterModerator["ScatteringCrossSection"])
waterModerator["ModeratingRatio"] = ModeratingRatio(waterModerator["SlowingDownPower"], waterModerator["AbsorptionCrossSection"])

# Adding to heavy water
heavyWaterModerator["LogarithmicEnergyLoss"] = 0.510 # Cannot be calculated using formula above due to being a molecule
heavyWaterModerator["NumberOfCollisionsToThermalise"] = CollisionsToThermalise(heavyWaterModerator["LogarithmicEnergyLoss"])
heavyWaterModerator["SlowingDownPower"] = SlowingDownPower(heavyWaterModerator["LogarithmicEnergyLoss"], heavyWaterModerator["ScatteringCrossSection"])
heavyWaterModerator["ModeratingRatio"] = ModeratingRatio(heavyWaterModerator["SlowingDownPower"], heavyWaterModerator["AbsorptionCrossSection"])

# Adding to graphite
graphiteModerator["LogarithmicEnergyLoss"] = LogarithmicEnergyLoss(graphiteModerator["AtomicNumber"])
graphiteModerator["NumberOfCollisionsToThermalise"] = CollisionsToThermalise(graphiteModerator["LogarithmicEnergyLoss"])
graphiteModerator["SlowingDownPower"] = SlowingDownPower(graphiteModerator["LogarithmicEnergyLoss"], graphiteModerator["ScatteringCrossSection"])
graphiteModerator["ModeratingRatio"] = ModeratingRatio(graphiteModerator["SlowingDownPower"], graphiteModerator["AbsorptionCrossSection"])

# Creating the data frame and adding the values
moderation_df = pd.DataFrame(
    data = [[waterModerator["AbsorptionCrossSection"], waterModerator["ScatteringCrossSection"], waterModerator["LogarithmicEnergyLoss"], waterModerator["NumberOfCollisionsToThermalise"], waterModerator["SlowingDownPower"], waterModerator["ModeratingRatio"]],
            [heavyWaterModerator["AbsorptionCrossSection"], heavyWaterModerator["ScatteringCrossSection"], heavyWaterModerator["LogarithmicEnergyLoss"], heavyWaterModerator["NumberOfCollisionsToThermalise"], heavyWaterModerator["SlowingDownPower"], heavyWaterModerator["ModeratingRatio"]],
            [graphiteModerator["AbsorptionCrossSection"], graphiteModerator["ScatteringCrossSection"], graphiteModerator["LogarithmicEnergyLoss"], graphiteModerator["NumberOfCollisionsToThermalise"], graphiteModerator["SlowingDownPower"], graphiteModerator["ModeratingRatio"]]
           ],
    index = ["Water", "Heavy Water", "Graphite"],
    columns = ["Absorption C.S (m$^{-1}$)", "Scattering C.S (m$^{-1}$)", "Logarithmic Energy Loss", "Collisions To Thermalise", "Slowing Down Power (m$^{-1}$)", "Moderating Ratio"]
)

# Displaying the data frame
moderation_df

,Absorption C.S (m$^{-1}$),Scattering C.S (m$^{-1}$),Logarithmic Energy Loss,Collisions To Thermalise,Slowing Down Power (m$^{-1}$),Moderating Ratio
Water,2.200,165,0.927000,19.630569,152.955000,69.525000
Heavy Water,0.012,36,0.510000,35.681445,18.360000,1530.000000
Graphite,0.003,39,0.157769,115.342928,6.152991,2050.996868


### Selecting Coolant
To pick a good coolant, we want one that has a high specific heat capacity and low latent heat of vaporisation. Therefore we need to take this into account when selecting our coolant.

No functions are needed here just defining some variables so they can be compared. They will be added to a dictionary to make it easy to access and then added to a Pandas DataFrame.

In [13]:
#coolant information
lhvw = 2260 #kJ/kg latent heat of vaporisation of water at 20C 931 kJ/kg in a pressurizer
shcw = 4.184 #kJ/(kg*C) specific heat capacity of water at 20C
lhvhw = 2073.20 #kJ/kg latent heat of vaporisation of heavy water
shchw = 4.228 #kJ/(kg*C) specific heat capacity of heavy water at 20C
lhvco2 = 571.08 #kJ/kg latent heat of vaporisation of carbon dioxide
shcco2 = 0.84606 #kJ/(kg*C) specific heat capacity of water at 20C

waterCoolant = {
    "LHV" : 2260,
    "SHC" : 4.184
}

heavyWaterCoolant = {
    "LHV" : 2073.20,
    "SHC" : 4.228
}

carbonDioxideCoolant = {
    "LHV" : 571.08,
    "SHC" : 0.84606
}

coolant_df = pd.DataFrame(
    data = [[waterCoolant["LHV"], waterCoolant["SHC"]],
            [heavyWaterCoolant["LHV"], heavyWaterCoolant["SHC"]],
            [carbonDioxideCoolant["LHV"], carbonDioxideCoolant["SHC"]]
           ],
    index = ["Water", "Heavy Water", "Carbon Dioxide"],
    columns = ["Latent Heat of Vaporisation (kJ kg$^{-1}$)", "Specfic Heat Capacity (kJ kg$^{-1}$ K$^{-1}$)"]
)
coolant_df

,Latent Heat of Vaporisation (kJ kg$^{-1}$),Specfic Heat Capacity (kJ kg$^{-1}$ K$^{-1}$)
Water,2260.00,4.18400
Heavy Water,2073.20,4.22800
Carbon Dioxide,571.08,0.84606


### Nuclear calculations

The following cell defines all the functions and variables needed for the nuclear calculations. 

First a function is defined to find the Q value due to the induced fission of a U-235 nuclei to two daughter nuclei with atomic masses Md1 and Md2. This induced fission follows the decay equation

$ ^{235}_{92}\text{U} + ^1_0 \text{n} \to X + Y + ^3_0 \text{n}$

where X and Y are atoms that must have proton and neutron numbers chosen carefully so that baryon number is conserved.

The atomic masses for the sixteen most likely decay products were considered, and these masses are stored in the file called 'atomic masses.csv', and the corresponding Q values were found from these masses. The layout of the .csv file goes:

Decay 1 Daughter 1\
Decay 1 Daughter 2\
Decay 2 Daughter 1\
Decay 2 Daughter 2\
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;$\vdots$\
Decay 16 Daughter 1\
Decay 16 Daughter 2

The probability of each of these decays can be found in the 'probs.csv' file. As we have only taken the 16 most probable reactions, their total probability will not add up to 1, as is required by a probability distribution, so we have had to normalise these probabilities so that their sum equals 1.

The next function is the function that calculates the power output of the nuclear reactions. It is called each frame inside the game loop. It calculates the amount of energy released from a chain reaction of depth 12 and then adds this to the total amount of energy. The power output is multiplied by a scale factor to get it to the power output of a typical nuclear fission reactor. This scale factor also includes the factors of the nuclear reactor such as the amount of energy required to heat the coolant and also vaporise it as well as the efficiencies of all the heat transfers. We have had to multiply our power output by a scale factor because modelling a proper chain reaction is incredibly commputationally challenging, with just the depth of 12 we have considered requiring 1s to carry out the calculations, and anything above a depth of 16 took more than 30 seconds to complete, with the processing time increasing exponentially. 

The final function takes the current power output of the reactor and uses that to determine whether the reaction is stable, unstable or dying out.

In [ ]:
def qvalue(Md1, Md2):
    '''
    Function that calculates Q-Value in Mev. Mp=mass of parent nucleus, Mn mass of neutron Md1,Md2=mass of daughter nuclei. masses in terms of Mevc^2
    '''
    u=931.5 # MeV/c2
    neutronMass=1.008 # u
    uraniumMass=235.043928 # u
    Q=((uraniumMass + neutronMass)- (Md1 + Md2 + 3*neutronMass))*u
    return Q

#loading masses and determines q value for each possible reaction, adding these to an array
am = np.loadtxt('data/atomic masses.csv') # In steps of 2 as each pair of decay products are on separate rows
qvalues=[]
for i in range (0,31,2):
    values=qvalue(am[i],am[i+1])
    qvalues.append(values*1.6E-13) # Changes unit from Mev to Joules

    
#loading normalised probabilities for each possible reaction and calculating normalised probabilities
probabilities=np.loadtxt('data/probs.csv')
normconst=1/(np.sum(probabilities))
normprobabilities=probabilities*normconst

def CalculatePowerOutput(insertionPercentage, reactionDepth = 12):
    '''
    Loops through a decay chain of a depth of 12. A random energy is generated for each induced fission in this chain and a total energy is summed for each step. 
    This energy is then scaled up to save time on the computation and get it to be on the right order of magnitude as a normal nuclear reactor. 
    The power is then multiplied by a reciprocal model to simulate the control rods being inserted.
    '''
    
    totalE = 0
    currentReaction = 0
    stepE = 0 # The energy of each step
    energyScaleFactor = 1e7 # Scale factor chosen to account for energy lost due to heating and efficiency of turbine and to boost power output to expected level. 
    energyArray = []
    
    while currentReaction < reactionDepth:
        for energy in range(2**currentReaction):
            sumE = np.random.choice(qvalues, p=normprobabilities)
            stepE += sumE
        currentReaction += 1

    insertionScaleFactor = 0.01/(insertionPercentage/100 + 0.01)
    
    totalE += stepE * energyScaleFactor * insertionScaleFactor
    energyArray.append(totalE)
    
    powerOutput = np.sum(energyArray)/len(energyArray)
    
    return powerOutput

def DetermineStability(powerOutput, stabilityLimits = [0.7, 1.4]):
    if powerOutput > stabilityLimits[1]:
        return "Unstable"
    elif powerOutput < stabilityLimits[0]:
        return "Dying out" 
    else:
        return "Stable"

### Pygame
This is by far the largest part of the program so we will split the explaination into parts:
1. Displaying different screens
2. Buttons
3. Explaining the control rods (how they are drawn on screen, how they move etc)
4. Changing the power output
5. What does each screen show

##### Displaying different screens

The main premise behind different screens in Pygame is to have different surfaces that you blit (render) onto the screen. Although simply blitting different surfaces onto the screen would suffice in most cases, we need there to be three completely different menus so we deemed it important to have different procedures (not functions as they do not return a value) that would each represent a different screen. This would allow three seperate game loops that all use the same window to show what we need it to show. When a certain button is pressed, the appropriate function is called to switch the game loop that is running.

##### Buttons

The buttons in the code are created using Pygame rectangles and surfaces. Their size and colour are determined using a surface, and then the rectangle of this surface is then got using the .get_rect() method, and the rectangle is then used to set the position of the button. This places the buttons on the screen but so far collisions are not checked. Collisions are checked inside the for loop 
```python
for event in pygame.event.get():
    ...
```
This loop runs through each event that is happening in the current frame. If the user presses left click then the button needs to be checked if it is clicked

```python
for event in pygame.event.get():
    if event.type == pygame.MOUESBUTTONDOWN:
        ...
```

However, this code alone doesn't check whether the buttons have been clicked as this just checks if the left button has been pressed down on the mouse. To correct this we need to get the cursor position of the user, using the pygame method ```pygame.mouse.get_pos()```, and check that it is colliding with a rectangle, using the pygame method ```rect.collidepoint(coordinates)``` where the method returns true if coordinates are inside the rect.

```python
for event in pygame.event.get():
    if event.type == pygame.MOUESBUTTONDOWN:
        mousePos = pygame.mouse.get_pos()
            if rect.collidepoint(mousePos):
                function()
```

This allows the appropriate function to be called when a certain button is pressed just by ensuring that each button has its own unique rectangle.

##### The control rods

The control rods are the most interactive part of this project. They are all objects of the ControlRod class, belonging to the sprite group ControlRodGroup. The control rods are added to this group in a for loop, to ensure that there are a constant number of control rods that can be changed by the variable NUMBER_OF_CONTROL_RODS. Their position on the screen is determined by the position of the first control rods top left corner. An offset is added onto the x coordinate depending upon which control rod is being placed on the screen (the control rod on the furthest right has the largest offset).

The insertion percentage of the control rods is modified using the buttons at the top right. These allow the insertion percentage to be changed in increments of 0.1%, 1.0% and 5.0%. This is done by first checking if the movement will take the control rods outside of the allowed range of 0-100 % insertion. If the move is allowed, then all control rods in the ControlRodGroup sprite group are looped over, and all have the MoveUp or MoveDown method called depending upon which button is pressed and the appropriate relativeShift is passed as a parameter. These methods work by moving the control rods vertically by relativeShift*height each time a button is pressed. 

If this was to reimplemented, the insertion depth only needs to be checked once outside the ControlRod class, to ensure that each function/procedure has only one job. Another bug is that when the user presses the return button whilst the control rods are inserted and then presses start again, the insertion percentage at the top goes back to 0% but the control rods stay inserted. This is a relatively simple fix and the code will just need some refactoring to prevent this in future.

##### Changing the power output

The power output from the nuclear calculations is scaled up by a scale factor to get it to the appropriate power output of a nuclear reactor. Due to not fully understanding how the power output changes as a function of insertion depth, we decided to create our own model.

$$ P = \frac{0.01}{x + 0.01} $$

Where P is the scale factor that multiplies the power output and x is the total fraction of the control rods inserted into the fuel rods, ranging between 0 and 1. The factor of 0.01 appears in the numerator so as to scale the power output down. The factor of +0.01 in the denominator appears so that when $x=0$, $P$ does not tend to $\infty$, instead $P=1$ giving the maximum power output.

##### What does each screen show
Screen one was the start screen and this allows the user to select the moderator that they want to use. The user cannot press the start button to move onto the next section until they have first selected a moderator. The user is indicated as to which moderator they have chosen by the colour of the button they have pressed changing colour.

Screen two is the bulk of the code and is the visualisation tool. It is reached by pressing the start button on the start screen once a moderator has been chosen. The insertion depth of the control rods can be changed by pressing the buttons labelled. The power output and stability can be seen on the mid right and the return button can be pressed to go back to the start screen. If the user needs any help they can press the help button and see the help screen.

Screen three is the help screen and it just simply tells the user what happens in a nuclear reaction as well as a nuclear fission reactor, describing the components in detail.

In [ ]:
# Initialising Pygame and required parts of pygame
pygame.init()
pygame.font.init()
clock = pygame.time.Clock()

# Creates the screen surface and sets it to display width and height
screen = pygame.display.set_mode((DISPLAY_WIDTH,DISPLAY_HEIGHT))
pygame.display.set_caption("Fission Reactor Simulator")

# Loading the images
helpBackgroundImage = pygame.image.load("imgs/Uranium235_Diagram.jpg").convert()
fissionReactorImage = pygame.image.load("imgs/Fission_Diagram2.png").convert()

# Changing the size of the reactor image
fissionReactorImage = pygame.transform.scale(fissionReactorImage, (896, 504))
fissionReactorRect = fissionReactorImage.get_rect(midbottom=(DISPLAY_WIDTH/2, DISPLAY_HEIGHT))

# Creating the control rod sprite group
ControlRodGroup = pygame.sprite.Group()

# Adding the control rods to the sprite group
for i in range(0, NUMBER_OF_CONTROL_RODS):
    controlRod = ControlRod(CONTROL_ROD_WIDTH, CONTROL_ROD_HEIGHT, (352 + i*CONTROL_ROD_OFFSET, 110))
    ControlRodGroup.add(controlRod)

# Creating the fuel rod sprite group
FuelRodGroup = pygame.sprite.Group()

# Adding fuel rods to sprite group
for i in range(0, NUMBER_OF_FUEL_RODS):
    fuelRod = FuelRod(FUEL_ROD_WIDTH, FUEL_ROD_HEIGHT, (340 + i*FUEL_ROD_OFFSET, 370))
    FuelRodGroup.add(fuelRod)

# Creating the pygame font object
titleFont = pygame.font.SysFont('arialblack', 36)
helpButtonFont = pygame.font.SysFont('arialblack', 24)
controlRodButtonFont = pygame.font.SysFont('arialblack', 16)

# Start screen text
titleText = titleFont.render("Fission Reactor Simulator", False, BLACK)

# Defining the help text objects
helpText = helpButtonFont.render("Help", False, BLACK)
backText = helpButtonFont.render("Back", False, BLACK)
visualiserReturnText = helpButtonFont.render("Return", False, BLACK)
startText = helpButtonFont.render("Click to Start", False, BLACK)
waterText = helpButtonFont.render("Water", False, BLACK)
heavyWaterText = helpButtonFont.render("Heavy Water", False, BLACK)
graphText = helpButtonFont.render("Graphite", False, BLACK)

# Defining the control rod text objects for each button
controlRodDescriptiveText = helpButtonFont.render("Change Insertion Depth:", False, BLACK)
minusX0_1Text = controlRodButtonFont.render("-0.1%", False, BLACK)
minusX1_0Text = controlRodButtonFont.render("-1.0%", False, BLACK)
minusX5_0Text = controlRodButtonFont.render("-5.0%", False, BLACK)
plusX0_1Text = controlRodButtonFont.render("+0.1%", False, BLACK)
plusX1_0Text = controlRodButtonFont.render("+1.0%", False, BLACK)
plusX5_0Text = controlRodButtonFont.render("+5.0%", False, BLACK)

# Getting the rectangles of text objects and setting their center at the given coordinates
titleTextRect = titleText.get_rect(center = (DISPLAY_WIDTH/2, DISPLAY_HEIGHT/4))

helpTextRect = helpText.get_rect(center = helpButtonRect.center)
backTextRect = backText.get_rect(center = helpButtonRect.center)
visualiserReturnTextRect = visualiserReturnText.get_rect(center = visualiserReturnButtonRect.center)
startTextRect = startText.get_rect(center =startButtonRect.center)
waterTextRect = waterText.get_rect(center = waterButtonRect.center)
heavyWaterTextRect = heavyWaterText.get_rect(center = heavyWaterButtonRect.center)
graphTextRect = graphText.get_rect(center = graphButtonRect.center)

controlRodDescriptiveTextRect = controlRodDescriptiveText.get_rect(topleft=(0,0))
minusX0_1TextRect = minusX0_1Text.get_rect(center = controlButtonRect_minus_0_1.center)
minusX1_0TextRect = minusX1_0Text.get_rect(center = controlButtonRect_minus_1_0.center)
minusX5_0TextRect = minusX5_0Text.get_rect(center = controlButtonRect_minus_5_0.center)
plusX0_1TextRect = plusX0_1Text.get_rect(center = controlButtonRect_plus_0_1.center)
plusX1_0TextRect = plusX1_0Text.get_rect(center = controlButtonRect_plus_1_0.center)
plusX5_0TextRect = plusX5_0Text.get_rect(center = controlButtonRect_plus_5_0.center)

# Creating the stability box
stabilityBoxSurf = pygame.Surface(stabilityBoxDimensions)
stabilityBoxRect = stabilityBoxSurf.get_rect(topleft = stabilityBoxCoords)
stabilityBoxSurf.fill(stabilityColours["Unstable"])

def Start(RUNNING, CHOSEN_FPS):
    '''
    The start screen procedure. Shows the user the start screen when run. Containts four buttons, three of which allow the user to select a moderator
    and one allows the user to press start.
    '''
    # Allows the user to select a moderator
    moderatorChoice = None

    # Defines the colours of the buttons
    WATER_BUTTON_COLOUR = BUTTON_COLOUR
    HEAVY_WATER_BUTTON_COLOUR = BUTTON_COLOUR
    GRAPH_BUTTON_COLOUR = BUTTON_COLOUR
    SELECTED_BUTTON_COLOUR = (60, 60, 60)

    # Game loop runs the whole time the window is open
    while RUNNING:
        for event in pygame.event.get(): # Loops through all possible events
            if event.type == pygame.QUIT: # If the quit button is pressed, close pygame and exit the program.
                running = False
                pygame.quit()
                sys.exit()
                
            if event.type == pygame.MOUSEBUTTONDOWN: # If the mouse button is pressed down
                mousePos = pygame.mouse.get_pos() # Gets the current mouse position
                if startButtonRect.collidepoint(mousePos): # If the help button is pressed down whilst over the help button, switch the window to the visualiser screen
                    if moderatorChoice !=None:
                        # This if statement prevents the user from pressing start without having selected a moderator
                        Visualiser(RUNNING, CHOSEN_FPS, moderatorChoice)

                # If the water button is pressed, set moderator choice to Water and set the button colour accordingly 
                if waterButtonRect.collidepoint(mousePos):
                    moderatorChoice = 'Water'
                    WATER_BUTTON_COLOUR = SELECTED_BUTTON_COLOUR
                    HEAVY_WATER_BUTTON_COLOUR = BUTTON_COLOUR
                    GRAPH_BUTTON_COLOUR = BUTTON_COLOUR

                # If the heavy water button is pressed, set moderator choice to Water and set the button colour accordingly
                if heavyWaterButtonRect.collidepoint(mousePos):
                    moderatorChoice = 'Heavy water'
                    WATER_BUTTON_COLOUR = BUTTON_COLOUR
                    HEAVY_WATER_BUTTON_COLOUR = SELECTED_BUTTON_COLOUR
                    GRAPH_BUTTON_COLOUR = BUTTON_COLOUR

                # If the graphite water button is pressed, set moderator choice to Water and set the button colour accordingly
                if graphButtonRect.collidepoint(mousePos):
                    moderatorChoice = 'Graphite'
                    WATER_BUTTON_COLOUR = BUTTON_COLOUR
                    HEAVY_WATER_BUTTON_COLOUR = BUTTON_COLOUR
                    GRAPH_BUTTON_COLOUR = SELECTED_BUTTON_COLOUR
        
        # Filling screen
        screen.fill(START_SCREEN_BACKGROUND_COLOUR)

        # Title text
        screen.blit(titleText, titleTextRect)
                    
        # Start button with outline of 2 pixels
        pygame.draw.rect(screen, BUTTON_COLOUR, startButtonRect)
        pygame.draw.rect(screen, BUTTON_OUTLINE_COLOUR, startButtonRect, 2)
        screen.blit(startText, startTextRect)

        # Water button with outline of 2 pixels
        pygame.draw.rect(screen, WATER_BUTTON_COLOUR, waterButtonRect)
        pygame.draw.rect(screen, BUTTON_OUTLINE_COLOUR, waterButtonRect, 2)
        screen.blit(waterText, waterTextRect)

        #Heavy Water Button with outline of 2 pixels
        pygame.draw.rect(screen, HEAVY_WATER_BUTTON_COLOUR, heavyWaterButtonRect)
        pygame.draw.rect(screen, BUTTON_OUTLINE_COLOUR, heavyWaterButtonRect, 2)
        screen.blit(heavyWaterText, heavyWaterTextRect)

        #Graphite button with outline of 2 pixels
        pygame.draw.rect(screen, GRAPH_BUTTON_COLOUR, graphButtonRect)
        pygame.draw.rect(screen, BUTTON_OUTLINE_COLOUR, graphButtonRect, 2)
        screen.blit(graphText, graphTextRect)

        # Update the screen and ensure the window runs at set fps
        pygame.display.flip()
        clock.tick(CHOSEN_FPS)

def Visualiser(RUNNING, CHOSEN_FPS, MODERATOR_TYPE):
    '''
    The first window that is shown to the user after pressing the help button. The help button can be pressed to give the user some information about the visualisation.
    The insertion depth of the control rods can be changed using the buttons in the top left. The power output and stability are shown on the right.
    '''

    insertion = 0 # Changes with the insertion percentage of the fuel rods.

    # Creates text to show the chosen moderator type at the top of the screen
    moderaterTypeText = helpButtonFont.render(f"Moderator: {MODERATOR_TYPE}", False, BLACK)
    moderaterTypeTextRect = moderaterTypeText.get_rect(midtop = (DISPLAY_WIDTH/2, 0))
    
    # Creates text to show the current insertion percentage at the top of the screen
    currentInsertionText = helpButtonFont.render(f"Current Insertion: {round(insertion,1)} %", False, BLACK)
    currentInsertionRect = currentInsertionText.get_rect(midtop = (DISPLAY_WIDTH/2, 20))

    # Defines the required stuff for the turbine such as the image and scaling it down, the turbine coordinates and the rotation speed.
    turbineCoords = (775, 315)
    originalTurbineImage = pygame.image.load("imgs/Turbine image.png").convert_alpha()
    originalTurbineImage = pygame.transform.scale(originalTurbineImage, (110,110))
    turbineRect = originalTurbineImage.get_rect(center = turbineCoords)
    originalTurbineImage.set_colorkey((255,255,255))
    angle = 0
    timePeriod = 2
    angleStep = 360 / (timePeriod * CHOSEN_FPS)

    # Game loop that runs the whole time the window is open
    while RUNNING:
        for event in pygame.event.get(): # Loops through all possible events
            if event.type == pygame.QUIT: # If the quit button is pressed, close pygame and exit the program.
                RUNNING = False
                pygame.quit()
                sys.exit
                
            if event.type == pygame.MOUSEBUTTONDOWN: # If the mouse button is pressed down      
                mousePos = pygame.mouse.get_pos() # Gets the current mouse position
                if helpButtonRect.collidepoint(mousePos): # If the help button is pressed down whilst over the help button, switch the window to the Help screen
                    Help(RUNNING, CHOSEN_FPS, MODERATOR_TYPE)

                # Checks which control rod button is pressed and do the appropriate move according to the one pressed given it will not take the control rods outside the range 0-100
                if controlButtonRect_minus_0_1.collidepoint(mousePos):
                    if insertion - 0.1 >= 0:
                            insertion -= 0.1
                    for controlRod in ControlRodGroup:
                        controlRod.MoveUp(-0.1)
                if controlButtonRect_minus_1_0.collidepoint(mousePos):
                    if insertion - 1.0 >= 0:
                            insertion -= 1.0
                    for controlRod in ControlRodGroup:
                        controlRod.MoveUp(-1.0)
                if controlButtonRect_minus_5_0.collidepoint(mousePos):
                    if insertion - 5.0 >= 0:
                            insertion -= 5.0
                    for controlRod in ControlRodGroup:
                        controlRod.MoveUp(-5.0)
                if controlButtonRect_plus_0_1.collidepoint(mousePos):
                    if insertion + 0.1 <= 100:
                            insertion += 0.1
                    for controlRod in ControlRodGroup:
                        controlRod.MoveDown(0.1)
                if controlButtonRect_plus_1_0.collidepoint(mousePos):
                    if insertion + 1.0 <= 100:
                            insertion += 1.0
                    for controlRod in ControlRodGroup:
                        controlRod.MoveDown(1.0)
                if controlButtonRect_plus_5_0.collidepoint(mousePos):
                    if insertion + 5.0 <= 100:
                            insertion += 5.0
                    for controlRod in ControlRodGroup:
                        controlRod.MoveDown(5.0)

                
                # Changes the insertion text to show the correct insertion percentage
                currentInsertionText = helpButtonFont.render(f"Current Insertion: {round(insertion, 1)} %", False, BLACK)

                # If the return button is pressed, go back to the start screen.
                if visualiserReturnButtonRect.collidepoint(mousePos):
                    Start(RUNNING, CHOSEN_FPS)
                    
        # Gets the current power output for the insertion and makes the stability box the correct colour and have the right text
        powerOutput = CalculatePowerOutput(insertion)
        stabilityText = DetermineStability(powerOutput)
        stabilityBoxSurf.fill(stabilityColours[stabilityText])
        stabilityText = controlRodButtonFont.render(f"{stabilityText}", False, BLACK)
        stabilityTextRect = stabilityText.get_rect(center = stabilityBoxRect.center)

        # Dsiaplys the power output text on the screen to 2 decimal places.
        powerOutputText = controlRodButtonFont.render(f"{powerOutput:.2f} GW", False, BLACK)
        powerOutputRect = powerOutputText.get_rect(center = (1000, 310))
    
        # Filling screen
        screen.fill(BACKGROUND_COLOUR)

        # Drawing the reactor image
        screen.blit(fissionReactorImage, fissionReactorRect)
        screen.blit(currentInsertionText, currentInsertionRect)

        # Drawing the stability rectangle and text
        screen.blit(stabilityBoxSurf, stabilityBoxRect)
        screen.blit(stabilityText, stabilityTextRect)
        
        # Rotate the turbine image
        angle += angleStep * (1.1 - insertion/100) # with no insertion there is roughly 1 rotation every 2 seconds, with full insertion there is one rotation roughly every 30 seconds.
        turbineImage = pygame.transform.rotate(originalTurbineImage, angle)
        turbineImage.set_colorkey((255,255,255))
        turbineRect = turbineImage.get_rect(center = turbineCoords)

        # Drawing the turbine image
        screen.blit(turbineImage , turbineRect)

        # Displaying the power output text
        screen.blit(powerOutputText, powerOutputRect)

        # Help button
        pygame.draw.rect(screen, BUTTON_COLOUR, helpButtonRect)
        pygame.draw.rect(screen, BUTTON_OUTLINE_COLOUR, helpButtonRect, 2)
        screen.blit(helpText, helpTextRect)

        # Return button
        pygame.draw.rect(screen, BUTTON_COLOUR, visualiserReturnButtonRect)
        pygame.draw.rect(screen, BUTTON_OUTLINE_COLOUR, visualiserReturnButtonRect, 2)
        screen.blit(visualiserReturnText, visualiserReturnTextRect)

        # Control rod descriptive text
        screen.blit(controlRodDescriptiveText, controlRodDescriptiveTextRect)

        # Chosen moderator type
        screen.blit(moderaterTypeText, moderaterTypeTextRect)
        
        # -0.1x button
        pygame.draw.rect(screen, CONTROL_BUTTON_COLOUR_MINUS_0_1x, controlButtonRect_minus_0_1)
        pygame.draw.rect(screen, BUTTON_OUTLINE_COLOUR, controlButtonRect_minus_0_1, 2)
        screen.blit(minusX0_1Text, minusX0_1TextRect)

        # -1.0x button
        pygame.draw.rect(screen, CONTROL_BUTTON_COLOUR_MINUS_1_0x, controlButtonRect_minus_1_0)
        pygame.draw.rect(screen, BUTTON_OUTLINE_COLOUR, controlButtonRect_minus_1_0, 2)
        screen.blit(minusX1_0Text, minusX1_0TextRect)

        # -5.0x button
        pygame.draw.rect(screen, CONTROL_BUTTON_COLOUR_MINUS_5_0x, controlButtonRect_minus_5_0)
        pygame.draw.rect(screen, BUTTON_OUTLINE_COLOUR, controlButtonRect_minus_5_0, 2)
        screen.blit(minusX5_0Text, minusX5_0TextRect)

        # +0.1x button
        pygame.draw.rect(screen, CONTROL_BUTTON_COLOUR_PLUS_0_1x, controlButtonRect_plus_0_1)
        pygame.draw.rect(screen, BUTTON_OUTLINE_COLOUR, controlButtonRect_plus_0_1, 2)
        screen.blit(plusX0_1Text, plusX0_1TextRect)

        # +1.0x button
        pygame.draw.rect(screen, CONTROL_BUTTON_COLOUR_PLUS_1_0x, controlButtonRect_plus_1_0)
        pygame.draw.rect(screen, BUTTON_OUTLINE_COLOUR, controlButtonRect_plus_1_0, 2)
        screen.blit(plusX1_0Text, plusX1_0TextRect)

        # +5.0x button
        pygame.draw.rect(screen, CONTROL_BUTTON_COLOUR_PLUS_5_0x, controlButtonRect_plus_5_0)
        pygame.draw.rect(screen, BUTTON_OUTLINE_COLOUR, controlButtonRect_plus_5_0, 2)
        screen.blit(plusX5_0Text, plusX5_0TextRect)

        # Draws the control rods to the surface called screen
        ControlRodGroup.draw(screen)
        FuelRodGroup.draw(screen)

        # Update the screen and lock the framerate to the specified fps
        pygame.display.flip()
        clock.tick(CHOSEN_FPS)

def Help(RUNNING, CHOSEN_FPS, MODERATOR_TYPE):
    # Game loop that runs the whole time the window is open
    while RUNNING:
        for event in pygame.event.get(): # Loops through all possible events
            if event.type == pygame.QUIT: # If the quit button is pressed, close pygame and exit the program.
                running = False
                pygame.quit()
                sys.exit()
                
            if event.type == pygame.MOUSEBUTTONDOWN: # If the mouse button is pressed down
                mousePos = pygame.mouse.get_pos() # Gets the current mouse position
                if helpButtonRect.collidepoint(mousePos): # If the help button is pressed down whilst over the help button, switch the window to the Start screen
                    Visualiser(RUNNING, CHOSEN_FPS, MODERATOR_TYPE)
        
        # Filling screen
        screen.blit(helpBackgroundImage, (0,0))
                    
        # This is now the return button of the help screen. The same button has been used as it is in the same place, just the text is different.
        pygame.draw.rect(screen, RED, helpButtonRect)
        pygame.draw.rect(screen, BUTTON_OUTLINE_COLOUR, helpButtonRect, 2)
        screen.blit(backText, backTextRect)
        
        pygame.display.flip()
        clock.tick(CHOSEN_FPS)

Start(RUNNING, CHOSEN_FPS)